# 07 Diffuse material

* 난반사 재질을 구현합니다.

In [1]:
from typing import List
import sys
from tqdm import tqdm

import numpy as np
import numpy.linalg as LA

import matplotlib.pyplot as plt

infinity = sys.float_info.max

In [2]:
def normalize(vec: np.array, eps: float = 1e-6):
    return vec / (LA.norm(vec) + eps)


def blend(color1: np.array, color2: np.array, t: float):
    return (1.0 - t) * color1 + t * color2


def linear_to_gamma(linear_component: np.array) -> np.array:
    return np.where(linear_component > 0, np.sqrt(linear_component), linear_component)


def scale_color(color: np.array, num_sample_per_pixel: int) -> np.array:
    # divide the color by the number of samples and gamma-correct for gamma=2.0.
    scale = 1.0 / num_sample_per_pixel
    rgb = scale * color
    rgb = np.clip(rgb, 0.0, 0.999)
    return rgb

In [3]:
class Interval:
    def __init__(self, min: float, max: float):
        self.min = min
        self.max = max

    def size(self) -> float:
        return self.max - self.min

    def contains(self, x: float) -> bool:
        return self.min <= x <= self.max

    def surrounds(self, x: float) -> bool:
        return self.min < x < self.max

    def clamp(self, x: float) -> float:
        if x < self.min:
            return self.min
        if x > self.max:
            return self.max
        return x


empty_interval = Interval(min=infinity, max=-infinity)
universe_interval = Interval(min=-infinity, max=infinity)

In [4]:
class Ray:
    def __init__(self, origin: np.array, direction: np.array) -> None:
        self.origin: np.array = origin
        self.direction: np.array = normalize(direction)

    def at(self, t: float) -> np.array:
        return self.origin + t * self.direction

In [5]:
class HitRecord:
    def __init__(self, point: np.array, normal: np.array, t: float) -> None:
        self.point = point
        self.normal = normal
        self.t = t

    def set_face_normal(self, ray: Ray, outward_normal: np.array) -> None:
        is_front_face = np.dot(ray.direction, outward_normal) < 0
        self.normal = outward_normal if is_front_face else -outward_normal

In [6]:
class Hittable:
    def hit(self, ray: Ray, ray_t: Interval) -> HitRecord:
        raise NotImplementedError

In [7]:
class Sphere(Hittable):
    def __init__(self) -> None:
        self.center = np.zeros(3)
        self.radius = 1.0

    def __init__(self, center: np.array, radius: float) -> None:
        self.center = center
        self.radius = radius

    def hit(self, ray: Ray, ray_t: Interval) -> bool:
        dir_center_to_origin = ray.origin - self.center

        a = np.dot(ray.direction, ray.direction)
        half_b = np.dot(dir_center_to_origin, ray.direction)
        c = np.dot(dir_center_to_origin, dir_center_to_origin) - self.radius**2.0

        discriminant = half_b**2 - a * c
        if discriminant < 0:
            return None
        sqrt_d = np.sqrt(discriminant)

        # find the nearest root that lies in the acceptable range.
        t = (-half_b - sqrt_d) / a
        if not ray_t.surrounds(t):
            t = (-half_b + sqrt_d) / a
            if not ray_t.surrounds(t):
                return None

        point = ray.at(t)
        outward_normal = (point - self.center) / self.radius
        hit_record = HitRecord(point=point, normal=outward_normal, t=t)
        hit_record.set_face_normal(ray=ray, outward_normal=outward_normal)

        return hit_record

In [8]:
class HittableList(Hittable):
    def __init__(self) -> None:
        self.objects: List[Hittable] = []

    def clear(self) -> None:
        self.objects.clear()

    def add(self, object: Hittable) -> None:
        self.objects.append(object)

    def hit(self, ray: Ray, ray_t: Interval) -> HitRecord:
        record = None
        closest_so_far = ray_t.max

        for object in self.objects:
            tmp_record = object.hit(ray=ray, ray_t=Interval(ray_t.min, closest_so_far))
            if tmp_record is not None:
                closest_so_far = tmp_record.t
                record = tmp_record

        return record

In [9]:
class Camera:
    def __init__(self, image_width: int, image_height: int) -> None:
        self.aspect_ratio = 16.0 / 9.0
        self.viewport_height = 2.0
        self.viewport_width = self.aspect_ratio * self.viewport_height
        self.focal_length = 1.0

        self.origin = np.array([0.0, 0.0, 0.0])
        self.horizontal_vec = np.array([self.viewport_width, 0.0, 0.0])
        self.vertical_vec = np.array([0.0, self.viewport_height, 0.0])
        self.frontal_vec = np.array([0.0, 0.0, self.focal_length])
        self.lower_left_corner = (
            self.origin
            - self.horizontal_vec / 2.0
            - self.vertical_vec / 2.0
            - self.frontal_vec
        )

        self.pixel_delta_u = self.horizontal_vec / image_width
        self.pixel_delta_v = self.vertical_vec / image_height

    def get_ray(self, u: float, v: float) -> Ray:
        return Ray(
            origin=self.origin,
            direction=(
                self.lower_left_corner + u * self.pixel_delta_u + v * self.pixel_delta_v
            )
            - self.origin,
        )

In [10]:
def random_in_unit_sphere() -> np.array:
    p = np.random.uniform(low=-1.0, high=1.0, size=[3])
    return normalize(p)


def random_in_hemisphere(normal: np.array) -> np.array:
    p = random_in_unit_sphere()
    if np.dot(p, normal) > 0.0:
        return p
    else:
        return -p

In [11]:
def ray_color(ray: Ray, world: Hittable) -> np.array:
    record = world.hit(ray=ray, ray_t=Interval(0.0, infinity))
    if record is not None:
        direction = random_in_hemisphere(record.normal)
        new_ray = Ray(origin=record.point, direction=direction)
        return 0.5 * ray_color(ray=new_ray, world=world)

    t = 0.5 * (ray.direction[1] + 1.0)
    color1 = np.array([1.0, 1.0, 1.0])
    color2 = np.array([0.5, 0.7, 1.0])
    return blend(color1=color1, color2=color2, t=t)

In [12]:
# image
aspect_ratio = 16.0 / 9.0
image_width = 100
image_height = int(image_width / aspect_ratio)
num_sample_per_pixel = 100

image = np.zeros([image_height, image_width, 3])

In [13]:
# world
world = HittableList()
world.add(Sphere(center=np.array([0.0, 0.0, -1.0]), radius=0.5))
world.add(Sphere(center=np.array([0.0, -100.5, -1.0]), radius=100.0))

In [14]:
# camera
camera = Camera(image_width, image_height)

In [ ]:
%%time

for row in tqdm(range(image_height)):
  for col in range(image_width):
    for _ in range(num_sample_per_pixel):
      u = (col + np.random.uniform(low=0.0, high=1.0))
      v = (row + np.random.uniform(low=0.0, high=1.0))
      ray = camera.get_ray(u, v)
      image[row, col] += ray_color(ray=ray, world=world)

image = scale_color(image, num_sample_per_pixel=num_sample_per_pixel)

In [ ]:
plt.imshow(image, origin="lower")
plt.show()

### Limiting the Number of Child Rays

In [17]:
# Add depth parameter
def ray_color(ray: Ray, world: Hittable, depth: int) -> np.array:
    if depth <= 0:
        color = np.zeros(3)
        return color

    record = world.hit(ray=ray, ray_t=Interval(0.0, infinity))
    if record is not None:
        direction = random_in_hemisphere(record.normal)
        new_ray = Ray(origin=record.point, direction=direction)
        return 0.5 * ray_color(ray=new_ray, world=world, depth=depth - 1)

    t = 0.5 * (ray.direction[1] + 1.0)
    color1 = np.array([1.0, 1.0, 1.0])
    color2 = np.array([0.5, 0.7, 1.0])
    return blend(color1=color1, color2=color2, t=t)

In [18]:
# image
aspect_ratio = 16.0 / 9.0
image_width = 100
image_height = int(image_width / aspect_ratio)
num_sample_per_pixel = 100
max_depth = 5

image = np.zeros([image_height, image_width, 3])

In [ ]:
%%time

for row in tqdm(range(image_height)):
  for col in range(image_width):
    for _ in range(num_sample_per_pixel):
      u = (col + np.random.uniform(low=0.0, high=1.0))
      v = (row + np.random.uniform(low=0.0, high=1.0))
      ray = camera.get_ray(u, v)
      image[row, col] += ray_color(ray=ray, world=world, depth=max_depth)

image = scale_color(image, num_sample_per_pixel=num_sample_per_pixel)

In [ ]:
plt.imshow(image, origin="lower")
plt.show()

### Fixing Shadow Acne

In [29]:
def ray_color(ray: Ray, world: Hittable, depth: int) -> np.array:
    if depth <= 0:
        color = np.zeros(3)
        return color

    # fixing numerical error
    record = world.hit(ray=ray, ray_t=Interval(0.001, infinity))
    if record is not None:
        direction = random_in_hemisphere(record.normal)
        new_ray = Ray(origin=record.point, direction=direction)
        return 0.5 * ray_color(ray=new_ray, world=world, depth=depth - 1)

    t = 0.5 * (ray.direction[1] + 1.0)
    color1 = np.array([1.0, 1.0, 1.0])
    color2 = np.array([0.5, 0.7, 1.0])
    return blend(color1=color1, color2=color2, t=t)

In [ ]:
%%time

for row in tqdm(range(image_height)):
  for col in range(image_width):
    for _ in range(num_sample_per_pixel):
      u = (col + np.random.uniform(low=0.0, high=1.0))
      v = (row + np.random.uniform(low=0.0, high=1.0))
      ray = camera.get_ray(u, v)
      image[row, col] += ray_color(ray=ray, world=world, depth=max_depth)

image = scale_color(image, num_sample_per_pixel=num_sample_per_pixel)

In [ ]:
plt.imshow(image, origin="lower")
plt.show()

### True Lambertian Reflection

In [24]:
def ray_color(ray: Ray, world: Hittable, depth: int) -> np.array:
    if depth <= 0:
        color = np.zeros(3)
        return color

    record = world.hit(ray=ray, ray_t=Interval(0.001, infinity))
    if record is not None:
        # True Lambetian Reflection
        direction = record.normal + random_in_unit_sphere()
        new_ray = Ray(origin=record.point, direction=direction)
        return 0.5 * ray_color(ray=new_ray, world=world, depth=depth - 1)

    t = 0.5 * (ray.direction[1] + 1.0)
    color1 = np.array([1.0, 1.0, 1.0])
    color2 = np.array([0.5, 0.7, 1.0])
    return blend(color1=color1, color2=color2, t=t)

In [ ]:
%%time

for row in tqdm(range(image_height)):
  for col in range(image_width):
    for _ in range(num_sample_per_pixel):
      u = (col + np.random.uniform(low=0.0, high=1.0))
      v = (row + np.random.uniform(low=0.0, high=1.0))
      ray = camera.get_ray(u, v)
      image[row, col] += ray_color(ray=ray, world=world, depth=max_depth)

image = scale_color(image, num_sample_per_pixel=num_sample_per_pixel)

In [ ]:
plt.imshow(image, origin="lower")
plt.show()

### Gamma correction

In [32]:
image = linear_to_gamma(image)

In [ ]:
plt.imshow(image, origin="lower")
plt.show()